In [322]:
import json
import requests
from api_keys import g_key
import gmaps
import pandas as pd
import os
from matplotlib.cm import Reds
from matplotlib.colors import to_hex

gmaps.configure(api_key = g_key)

countries_string = requests.get(
    "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
).content
countries = json.loads(countries_string)

In [323]:
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(zoom_level = 2, center = (40,10), layout=figure_layout)
geojson = gmaps.geojson_layer(countries, fill_opacity=0.05, stroke_weight=2)
fig.add_layer(geojson)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='0px', wi…

In [324]:
csv_file = os.path.join('GDP_1970_to_2018.csv')
csv_file
gdp_data = pd.read_csv(csv_file)
gdp_data.head()

,Country Name,Country Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,AFG,0.174889,0.183111,0.159556,0.173333,0.215556,0.236667,0.255556,0.295333,...,1.585657,1.780428,2.000162,2.056105,2.048487,1.990711,1.936264,2.019176,1.936297,NaN
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.192696,1.289087,1.231978,1.277628,1.322824,1.138693,1.186135,1.302506,1.510250,NaN
2,Algeria,DZA,0.486349,0.507722,0.676179,0.871511,1.320971,1.555793,1.772835,2.097190,...,16.120507,20.001536,20.906289,20.975476,21.380881,16.597843,16.003293,16.739027,17.375795,NaN
3,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.057600,0.057400,0.064400,0.064100,0.064200,0.066100,0.065200,0.060200,0.063600,NaN
4,Andorra,AND,0.007862,0.008941,0.011341,0.015082,0.018656,0.022013,0.022728,0.025402,...,0.335570,0.344206,0.316462,0.328159,0.335074,0.281149,0.287731,0.301339,0.323654,NaN


In [325]:
gdp_in_1970 = gdp_data[['Country Name','Country Code','1970']]
gdp_in_1970.head()

,Country Name,Country Code,1970
0,Afghanistan,AFG,0.174889
1,Albania,ALB,NaN
2,Algeria,DZA,0.486349
3,American Samoa,ASM,NaN
4,Andorra,AND,0.007862


In [326]:
common_country_names = []
common_country_gdp = []

for feature in countries['features']:
    gmap_country_name = feature['properties']['name']
    if gmap_country_name in list(gdp_in_1970['Country Name']):
        common_country_names.append(gmap_country_name)
        common_country_gdp.append(list(gdp_in_1970[gdp_in_1970['Country Name'] == gmap_country_name]['1970']))

In [327]:
common_country_names[0]

'Afghanistan'

In [328]:
test_gdp = []
for _ in range(0,len(common_country_gdp)):
    test_gdp.append(common_country_gdp[_][0])

In [329]:
len(test_gdp)

147

In [330]:
test_dict = gdp_in_1970.to_dict('records')

In [331]:
gdp_list = []
for _ in range(0,147):
    temp_tuple = (common_country_names[_],test_gdp[_])
    gdp_list.append(temp_tuple)

In [332]:
len(gdp_list)

147

In [333]:
country2gdp = dict(gdp_list)

In [334]:
# We will need to scale the GDP values to lie between 0 and 1
min_gdp = min(country2gdp.values())
max_gdp = max(country2gdp.values())
gdp_range = max_gdp - min_gdp

def calculate_color(gdp_value):
    """
    Convert the GDP coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gdp = (gdp_value - min_gdp) / gdp_range

    # invert gini so that high inequality gives dark color
    inverse_gdp = 1.0 - normalized_gdp

    # transform the gini coefficient to a matplotlib color
    mpl_color = Reds(inverse_gdp)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [335]:
#  build an array of colors, one for each country
colors = []
for feature in countries['features']:
    country_name = feature['properties']['name']
    try:
        gdp = country2gdp[country_name]
        color = calculate_color(gdp)
    except KeyError:
        # no gdp for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)
#     counter = counter + 1

In [336]:
# fig = gmaps.figure()

figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(zoom_level = 2, center = (40,10), layout=figure_layout)
geojson = gmaps.geojson_layer(countries, fill_opacity=0.05, stroke_weight=2)
gdp_layer = gmaps.geojson_layer(countries,fill_color=colors, fill_opacity = 0.8, stroke_weight=2)
fig.add_layer(geojson)
fig

fig.add_layer(gdp_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='0px', wi…